# The main algorithm for regression in PEST is the Gauss-Levenberg-Marquardt (GLM)  algorithm



In [ ]:
%matplotlib inline
import os
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyemu
plt.rcParams['font.size']=10
import xsect_setup
import response_surface as resurf

copy the model files over

In [ ]:
xsect_setup.setup_model()

In [ ]:
working_dir = xsect_setup.WORKING_DIR
pst_name = xsect_setup.PST_NAME

## Let's open up the `PST` file and check out the parameters

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name))

In [ ]:
inpst.parameter_data

## We need to fix forecast flux and put reasonable bounds on the other parameters

In [ ]:
inpst.parameter_data.loc['hk1', 'parlbnd'] = 1.0
inpst.parameter_data.loc['hk1', 'parubnd'] = 25.0
inpst.parameter_data.loc['cal_flux', 'parlbnd'] = 0.001
inpst.parameter_data.loc['cal_flux', 'parubnd'] = 5
inpst.parameter_data.loc['fore_flux', 'partrans'] = 'fixed'
inpst.write(os.path.join(working_dir,pst_name))
inpst.parameter_data


## run the response surface function

In [ ]:
resurf.run_respsurf(['hk1', 'cal_flux'], pstfile=pst_name, WORKING_DIR=working_dir)

In [ ]:
fig, ax = resurf.plot_response_surface(parnames=['hk1', 'cal_flux'], 
                             pstfile=pst_name, WORKING_DIR=working_dir,
                            nanthresh = 10, maxresp=10.0);

## Let's evaluate the impact of Lambda on regression (regularization included)

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name))
# turn off the Maquardt lambda
inpst.pestpp_options['lambdas']=0.0
inpst.pestpp_options['lambda_scale_fac']=1.0
inpst.pestpp_options['upgrade_augment']='False'

inpst.parameter_data.loc['fore_flux', 'partrans'] = 'fixed'
# set the starting values up in the corner of parameter space
inpst.parameter_data.loc['hk1', 'parval1'] = 20.0
inpst.parameter_data.loc['cal_flux', 'parval1'] = 4.0
inpst.control_data.noptmax=250
inpst.write(os.path.join(working_dir,pst_name.replace('.pst','.nolam.pst')))

In [ ]:
os.chdir(working_dir)
pyemu.helpers.start_slaves('.', 'pestpp', 
                           pst_name.replace('.pst','.nolam.pst'), 
                           num_slaves=5, master_dir='.')
os.chdir("..")


In [ ]:
fig, ax = resurf.plot_response_surface(parnames=['hk1', 'cal_flux'], 
                             pstfile=pst_name, WORKING_DIR=working_dir,
                            nanthresh = 10, maxresp=10.0);
pars, obfun = resurf.add_trajectory_to_plot(fig,ax, 'nolam','No Lambda', working_dir, pst_name)

In [ ]:
obfun.total_phi.plot()

# Let's try a small value `(1.0)` for Lambda

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name.replace('.nolam.pst','.lam1.0.pst')))
# set the Maquardt lambda to 1.0
inpst.pestpp_options['lambdas']=1.0
inpst.pestpp_options['lambda_scale_fac']=1.0
inpst.pestpp_options['upgrade_augment']='False'

inpst.parameter_data.loc['hk1', 'parval1'] = 20.0
inpst.parameter_data.loc['cal_flux', 'parval1'] = 4.0
inpst.control_data.noptmax=250
inpst.write(os.path.join(working_dir,pst_name.replace('.pst','.lam1.0.pst')))

In [ ]:
os.chdir(working_dir)
pyemu.helpers.start_slaves('.', 'pestpp', 
                           pst_name.replace('.pst','.lam1.0.pst'), 
                           num_slaves=5, master_dir='.')
os.chdir("..")

In [ ]:
fig, ax = resurf.plot_response_surface(parnames=['hk1', 'cal_flux'], 
                             pstfile=pst_name, WORKING_DIR=working_dir,
                            nanthresh = 10, maxresp=10.0);
pars, obfun = resurf.add_trajectory_to_plot(fig,ax, 'lam1.0','Lambda = 1.0', working_dir, pst_name)

### This is RAD! Much better direction! It turns out, we can also do a line search along this better tragectory to 0) make use of idle computer time and 1) avoid overshooting. However, beyond that, it might even make sense to _intentionally_ overshoot a bit.

In [ ]:
obfun.total_phi.plot()

# Let's try a really big value value `(50.0)` for Lambda

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name.replace('.nolam.pst','.lam50.0.pst')))
# set the Maquardt lambda to 1.0
inpst.pestpp_options['lambdas']=50.0
inpst.pestpp_options['lambda_scale_fac']=1.0
inpst.pestpp_options['upgrade_augment']='False'

inpst.parameter_data.loc['hk1', 'parval1'] = 20.0
inpst.parameter_data.loc['cal_flux', 'parval1'] = 4.0
# let's make sure it keeps going even with tiny steps
inpst.control_data.noptmax=550
inpst.control_data.phiredstp=0.000001
inpst.control_data.relparstp=0.000001
inpst.control_data.nphistp=6
inpst.control_data.nrelpar=6
inpst.write(os.path.join(working_dir,pst_name.replace('.pst','.lam50.0.pst')))

In [ ]:
os.chdir(working_dir)
pyemu.helpers.start_slaves('.', 'pestpp', 
                           pst_name.replace('.pst','.lam50.0.pst'), 
                           num_slaves=5, master_dir='.')
os.chdir("..")

In [ ]:
fig, ax = resurf.plot_response_surface(parnames=['hk1', 'cal_flux'], 
                             pstfile=pst_name, WORKING_DIR=working_dir,
                            nanthresh = 10, maxresp=10.0);
pars, obfun = resurf.add_trajectory_to_plot(fig,ax, 'lam50.0','Lambda = 50.0', working_dir, pst_name)

In [ ]:
obfun.total_phi.plot()

In [ ]:
pars.tail()

In [ ]:
obfun.tail()